In [ ]:
# Panel B
import json
import matplotlib.pyplot as plt
rpneg = json.load(open("./HZV029_plasma_RP_neg_experiment.json"))
hilicpos = json.load(open("./HZV029_plasma_HILIC_pos_experiment.json"))

print(len(rpneg["acquisitions"]))
print(len(hilicpos["acquisitions"]))

for title, data in zip(["HZV029 RP-", "HZV029 HILIC+"], [rpneg, hilicpos]):
    cmds = data["command_history"]
    total_time_elapsed = {}
    for i, cmd_1 in enumerate(cmds[:-1]):
        cmd_2 = cmds[i+1]
        time_1, cmd_1 = cmd_1.split(":")
        time_2, cmd_2 = cmd_2.split(":")
        subcommand = cmd_2.split(";")[1]
        time_used = float(time_2) - float(time_1)
        if subcommand not in total_time_elapsed:
            total_time_elapsed[subcommand] = 0
        total_time_elapsed[subcommand] += time_used / 60
    print(total_time_elapsed.keys())
    colors = {
        'assemble': 'lightblue',
        'asari': 'lightblue',
        'convert': 'lightblue',
        'blank_masking': 'lightcoral',
        'drop_samples': 'lightcoral',
        'drop_outliers': 'lightcoral',
        'drop_missing_features': 'lightcoral',
        'normalize': 'lightcoral',
        'batch_correct': 'lightcoral', 
        'log_transform': 'lightcoral',
        'interpolate': 'lightcoral',
        'impute': 'lightcoral',
        'build_empCpds': 'lightgreen',
        'l4_annotate': 'lightgreen',
        'l2_annotate': 'lightgreen',
        'L1_annotate_w_MS2': 'lightgreen',
        'map_ms2': 'lightgreen',
        'l1a_annotate': 'lightgreen',
        'l1b_annotate': 'lightgreen',
        'report': 'lightgray',
        'finish': 'lightgreen'
    }


    subtotal = 0
    for sub, dt in total_time_elapsed.items():
        if sub not in {'finish'}:
            plt.bar(title, dt, bottom=subtotal, color=colors[sub])
            subtotal += dt
            print(sub, dt)
plt.show()
